In [1]:
import cv2
import mediapipe as mp
import inspect
import pyautogui
import numpy as np
import tensorflow as tf
from itertools import combinations

import keras 


2023-12-03 14:10:53.951036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
def load_model(dir:str):
    return_value= False
    if os.path.exists(dir):
        loaded_model = keras.models.load_model(dir)
        return_value= loaded_model
    else:
        return_value= False
    
    return return_value


def get_feature_hand_land_mark(coors_list:list)-> np.ndarray:
    feature_array= None
  
 #   target_node_muscle= [[0, 1, 2, 3, 4], [0, 5, 6, 7, 8], [0, 9, 10, 11, 12], [0, 13, 14, 15, 16], 
 #                        [0, 17, 18, 19, 20] ]
    target_node_muscle= [[0, 1, 2, 3, 4], [0, 5, 6, 7, 8], [0, 9, 10, 11, 12], [0, 13, 14, 15, 16], 
                         [0, 17, 18, 19, 20] , [1, 5, 9, 13, 17], [2, 6, 10, 14, 18], [3, 7, 11, 15, 19], 
                         [4, 8, 12, 16, 20]]
       
                    
    feature_array= []
    for nodes in target_node_muscle:
                      
                        coor_list= [coors_list[i] for i in nodes]

                        perm_list = combinations(coor_list, 2)
                        sub_feature_list= []
                        for i in perm_list:
                         
                            dist = [i[1][0] - i[0][0], i[1][1] - i[0][1]]
                          
                            norm = [np.sqrt(dist[0] ** 2 + dist[1] ** 2)]
                           
                          #  direction = [dist[0] / norm, dist[1] / norm]
                         #   bullet_vector = [direction[0] * np.sqrt(2), direction[1] * np.sqrt(2)]
                          
                            sub_feature_list+= norm
                        feature_array.append(sub_feature_list)
                       
                  
    feature_array= np.array(feature_array)  
    feature_array= np.expand_dims(feature_array, 0)
    print ('feature_array shape', feature_array.shape)
  
                  
    return feature_array

save_model_dir= os.path.join(os.getcwd(), 'detect_gesture_model_trans')
model= load_model(save_model_dir)


In [5]:

def scroll(index_finger_rel_x, index_finger_rel_y, middle_finger_rel_x, middle_finger_rel_y, ring_finger_rel_x, ring_finger_rel_y, pinky_finger_rel_x, pinky_finger_rel_y, previous_index_finger_rel_y, previous_middle_finger_rel_y):
    four_finers_connect_ratio_x= index_finger_rel_x/ middle_finger_rel_x/ ring_finger_rel_x/ pinky_finger_rel_x
    four_finers_connect_ratio_y= index_finger_rel_y/ middle_finger_rel_y/ ring_finger_rel_y/ pinky_finger_rel_y
    
    if middle_finger_rel_y- previous_middle_finger_rel_y> 0:
            pyautogui.scroll(-10) 
            print ('scroll down')
    else:
            pyautogui.scroll(10) 
            print ('scroll up')
        
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
num_seconds= 0.01
previous_index_finger_rel_y= 0

cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    previous_middle_finger_rel_y= None
    while cap.isOpened():
        
        success, image = cap.read()
        h, w, chn= image.shape
        frame_h= 899
        frame_w= 1499
        if not success:
            print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
            continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

    # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        previous_middle_finger_rel_x= None
        range_rate= 0.1

      #  label_name_dict= {'normal': 0, 'lclick': 1, 'scroll': 2, 'rclick':3}
       
        if results.multi_hand_landmarks:
               # print ('results.multi_hand_landmarks', results.multi_hand_landmarks)
                hand_landmarks= results.multi_hand_landmarks[0]
                
                coord_list= [(int(hand_landmarks.landmark[i].x*w), int(hand_landmarks.landmark[i].y* h)) for i in range(len(hand_landmarks.landmark))]
                
                prediction= model.predict(
                    get_feature_hand_land_mark(coord_list),
                    batch_size=None,
                    verbose="auto",
                    steps=None,
                    callbacks=None,
                    max_queue_size=10,
                    workers=1,
                    use_multiprocessing=False,
                )
               
                print ('predict gesture:', np.argmax(prediction))
             #   hand_landmarks_x= inspect.getmembers(hand_landmarks, lambda a:not(inspect.isroutine(x)))
                index_finger_rel_x= hand_landmarks.landmark[8].x
                index_finger_rel_y= hand_landmarks.landmark[8].y
                index_finger_pip_rel_x= hand_landmarks.landmark[6].x
                index_finger_pip_rel_y= hand_landmarks.landmark[6].y
                
                
                
                abs_x= frame_w- index_finger_rel_x* frame_w
                abs_y= index_finger_rel_y* frame_h
                z= hand_landmarks.landmark[8].z
                   
                middle_finger_rel_x= hand_landmarks.landmark[12].x
                middle_finger_rel_y= hand_landmarks.landmark[12].y
                
                thumb_tip_rel_x= hand_landmarks.landmark[4].x
                thumb_tip_rel_y= hand_landmarks.landmark[4].y
               
                pyautogui.moveTo(abs_x, abs_y, duration=num_seconds)
                if np.argmax(prediction)== 0:
            #        pyautogui.moveTo(abs_x, abs_y, duration=num_seconds)
                    continue
                if np.argmax(prediction)== 1:
                    dist= np.sqrt((index_finger_rel_x- thumb_tip_rel_x)** 2+ (index_finger_rel_y- thumb_tip_rel_y)** 2)
                    if dist<0.05:
                        print ('left click')
                        pyautogui.click()
                        
                    
               # elif np.argmax(prediction)==3:
                #    dist= np.sqrt((middle_finger_rel_x- thumb_tip_rel_x)** 2+ (middle_finger_rel_y- thumb_tip_rel_y)** 2)
               #     print ('dist', dist)
                    #if dist<0.06:
                   
                #        pyautogui.click(button='right')
                #        print ('right click')
                    
               # print (round(thumb_tip_rel_x/ index_finger_pip_rel_x, 2), round(thumb_tip_rel_y/ index_finger_pip_rel_y, 2))
                delta_thumb_tip_index_finger_pip= round((thumb_tip_rel_y/ index_finger_pip_rel_y)/ (thumb_tip_rel_x/ index_finger_pip_rel_x), 2)

                print ('delta_thumb_tip_index_finger_pip', delta_thumb_tip_index_finger_pip)
                
                if np.arange(thumb_tip_rel_y-range_rate, thumb_tip_rel_y+ range_rate).any()== np.arange(index_finger_pip_rel_y-range_rate, index_finger_pip_rel_y+ range_rate).any() and np.arange(thumb_tip_rel_x-range_rate, thumb_tip_rel_x+ range_rate).any()== np.arange(index_finger_pip_rel_x-range_rate, index_finger_pip_rel_x+ range_rate).any() and delta_thumb_tip_index_finger_pip* 100 in range(90, 100):
                    print ('thumb and index finger touch')
                # precision grip
               # if delta_thumb_tip_index_finger_pip* 100 in range(98, 103, 1):
               #     pyautogui.click()
               #     print ('click')
                
                ring_finger_rel_x= hand_landmarks.landmark[16].x
                ring_finger_rel_y= hand_landmarks.landmark[16].y
                pinky_finger_rel_x= hand_landmarks.landmark[20].x
                pinky_finger_rel_y= hand_landmarks.landmark[20].y


                delta_middle_finger_tip_index_finger_pip= round((middle_finger_rel_y/ index_finger_pip_rel_y)/ (middle_finger_rel_y/ index_finger_pip_rel_x), 2)
              #  print ('delta_thumb_tip_index_finger_pip', delta_thumb_tip_index_finger_pip*100)
              #  print ('delta_middle_finger_tip_index_finger_pip', delta_middle_finger_tip_index_finger_pip* 100)
                if np.argmax(prediction)== 2:
               # if delta_thumb_tip_index_finger_pip* 100 in range(90, 100, 1) and delta_middle_finger_tip_index_finger_pip*100 in range(45, 60, 1):
               #     print ('middle and index finger touch')
    
                    if previous_middle_finger_rel_y!= None:
                       
                        scroll(index_finger_rel_x, index_finger_rel_y, middle_finger_rel_x, middle_finger_rel_y, ring_finger_rel_x, ring_finger_rel_y, pinky_finger_rel_x, pinky_finger_rel_y, previous_index_finger_rel_y, previous_middle_finger_rel_y)

                mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
             #   previous_z= z
                previous_index_finger_rel_y= index_finger_rel_y
                previous_middle_finger_rel_y= middle_finger_rel_y

    # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        key= cv2.waitKey(5)
        if key == ord('q') or key == 27: # Esc
       
            break
     
      
cap.release()

feature_array shape (1, 9, 10)
1/1 [==============================] - 0s 16ms/step
predict gesture: 0
feature_array shape (1, 9, 10)
1/1 [==============================] - 0s 16ms/step
predict gesture: 0
feature_array shape (1, 9, 10)
1/1 [==============================] - 0s 22ms/step
predict gesture: 1
delta_thumb_tip_index_finger_pip 0.9
thumb and index finger touch
feature_array shape (1, 9, 10)
1/1 [==============================] - 0s 16ms/step
predict gesture: 1
left click
delta_thumb_tip_index_finger_pip 0.9
thumb and index finger touch
feature_array shape (1, 9, 10)
1/1 [==============================] - 0s 16ms/step
predict gesture: 2
delta_thumb_tip_index_finger_pip 0.98
thumb and index finger touch
scroll up
feature_array shape (1, 9, 10)
1/1 [==============================] - 0s 18ms/step
predict gesture: 1
left click
delta_thumb_tip_index_finger_pip 0.95
thumb and index finger touch
feature_array shape (1, 9, 10)
1/1 [==============================] - 0s 15ms/step
predict

KeyboardInterrupt: 